In [ ]:
CIFAR = '/content/drive/My Drive/Assignment/ML ASS 3/cifar-10-batches-py/'
import numpy as np

In [ ]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        cifar_dict = pickle.load(fo,encoding='bytes')
    return cifar_dict

In [ ]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i, direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR+direc)

In [ ]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [ ]:
all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
test_batch1 = [test_batch]
#all_train_batches = [data_batch1]
#all_train_batches1 = all_train_batches[:1000]
training_image = np.vstack([d[b"data"] for d in all_train_batches])
train_len = len(training_image)
training_image = training_image.reshape(train_len,3*32*32)
training_label = np.hstack([d[b"labels"] for d in all_train_batches])
training_label = training_label.reshape(50000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_image = np.vstack([d[b"data"] for d in test_batch1])
test_len = len(test_image)
test_image = test_image.reshape(test_len,3*32*32)
test_label = np.hstack([d[b"labels"] for d in test_batch1])
test_label = test_label.reshape(10000)



In [ ]:
train_image_new = training_image[:1000]
test_image_new = test_image[:1000]
train_new_label = training_label[:1000]
test_new_label = test_label[:1000]

In [ ]:
#from sklearn.preprocessing import StandardScaler 
#sc = StandardScaler() 
  
#X_train = sc.fit_transform(training_image) 
#X_test = sc.transform(test_image) 

In [14]:
from sklearn.decomposition import PCA

pca = PCA(.90)
X_train = pca.fit_transform(training_image)
X_test = pca.transform(test_image)


In [13]:
#import pickle
#filename = 'XtestPCA10000.sav'
#X_test = pickle.load(open(filename, 'rb'))

#filename = 'XtrainPCA50000.sav'
#X_train = pickle.load(open(filename, 'rb'))

UnpicklingError: ignored

In [15]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC 
import pickle 
# defining parameter range 
param_grid = {'C': [10],                 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1) 
  
# fitting the model for grid search 
grid.fit(X_train, training_label) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 25.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [10], 'kernel': ['rbf']}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=3)

In [16]:
print(grid.best_params_) 

{'C': 10, 'kernel': 'rbf'}


In [17]:
print(grid.best_estimator_) 

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [22]:
sv = grid.best_estimator_.fit(X_train,training_label)


In [23]:
sv.support_vectors_.shape

(43571, 99)

In [24]:
filename = '1-3 PCA SVM rbf C=10.sav'
pickle.dump(sv, open(filename, 'wb'))

In [25]:
X_train.shape

(50000, 99)

In [18]:
grid_trainpredict = grid.predict(X_train)

In [19]:
grid_predictions = grid.predict(X_test) 

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(training_label, grid_trainpredict))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      5000
           1       0.99      0.98      0.98      5000
           2       0.83      0.85      0.84      5000
           3       0.95      0.90      0.92      5000
           4       0.83      0.86      0.85      5000
           5       0.96      0.90      0.93      5000
           6       0.86      0.92      0.89      5000
           7       0.97      0.94      0.95      5000
           8       0.95      0.96      0.95      5000
           9       0.99      0.97      0.98      5000

    accuracy                           0.92     50000
   macro avg       0.92      0.92      0.92     50000
weighted avg       0.92      0.92      0.92     50000



In [21]:
print(classification_report(test_label, grid_predictions))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63      1000
           1       0.65      0.68      0.67      1000
           2       0.44      0.46      0.45      1000
           3       0.38      0.39      0.38      1000
           4       0.50      0.49      0.50      1000
           5       0.50      0.47      0.48      1000
           6       0.59      0.63      0.61      1000
           7       0.67      0.58      0.62      1000
           8       0.68      0.67      0.68      1000
           9       0.62      0.59      0.60      1000

    accuracy                           0.56     10000
   macro avg       0.56      0.56      0.56     10000
weighted avg       0.56      0.56      0.56     10000

